In [1]:
file = open('pr2d.txt')
text = file.read()
file.close()
text = text.split('\n')
verticeList = []
outverticeList=[]
text = [x for x in text if x != '']
for i in text: 
    sentence = i.split(' ')
    verticeList.append(str(sentence[0]))
    outverticeList.append(str(sentence[0]))
    verticeList.append(str(sentence[1]))
verticeList = list(set(verticeList))
outverticeList = list(set(outverticeList))
addvertice=[]
for i in verticeList:
    if i not in outverticeList:
        addvertice.append(i)
def splitVE(line):
    line = str(line).split(' ')
    ver = line[0]
    edge = [(line[1], float(line[2]))]
    return (ver, edge)

def fillINF(vertice):
    edges = vertice[1]
    wait = verticeList[:]
    for i in edges:
        wait.remove(i[0])
    for j in wait:
        if vertice[0] == j:
            vertice[1].append((j, 0))
        else:
            vertice[1].append((j, float("+inf")))
    m = vertice[1]
    m = sorted(m, key=lambda x: x[0])
    return (vertice[0],m)

def fillMIN(vertice, start):
    edge =vertice[1][:]
    ver = vertice[0]
    if start == ver:
        return (ver, (0, edge))
    else:
        return (ver, (float("+inf"), edge))



In [2]:
def getOriginalMatrix(data, start):
    """This function returns a complete path adjacency matrix of the start node
       Each line stores three parts:
       The first is a vertex name means 'from start to this vertex'
       Then the second part store the length from start to this vertex
       The final part stores from this vertex to all other vertices with the corresponding weight
       """
    graph = sc.textFile(data)
    formedGraph = graph.map(lambda x: splitVE(x))
    filledGraph = formedGraph.reduceByKey(lambda x, y: x+y) 
    filledINFGraph = filledGraph.map(lambda x: fillINF(x, data))
    filledLENGraph = filledINFGraph.map(lambda x: fillLENGTH(x, start))
    filledLENGraph.cache()
    return filledLENGraph

def getAdjDict(data, start):
    """This function calculates the dictionary within values of outward edges and weights,
       The key of this dictionary is the start
    """
    adj = getOriginalMatrix(data, start).collect()
    adjDict = dict()
    for i in adj:
        adjDict[i[0]] = i[1][1]
    return adjDict

In [3]:

graph = sc.textFile('pr2d.txt')
formedGraph = graph.map(lambda x: splitVE(x))
newrdd=[]
for i in addvertice:
    newrdd.append((i,[(i,0.0)]))
formedGraph.collect()  
newrdd=sc.parallelize(newrdd,4)
newrdd.cache()
formedGraph=formedGraph.union(newrdd)
#b=formedGraph.map(lambda x: fla(x)).filter(lambda x: len(x)>0).map(lambda x:(x[0][0],x))
#formedGraph.map(lambda x: fla(x)).collect()
#b=formedGraph.union(newrdd)
filledGraph = formedGraph.reduceByKey(lambda x, y: x+y) 
filledINFGraph = filledGraph.map(lambda x: fillINF(x))
filledMINGraph = filledINFGraph.map(lambda x: fillMIN(x, '1'))
filledMINGraph.cache()
#filledMINGraph.collect()
formedGraph.collect() 

[('2', [('3', 2.0)]),
 ('1', [('2', -3.0)]),
 ('1', [('5', 5.0)]),
 ('4', [('5', 2.0)]),
 ('3', [('4', 3.0)]),
 ('5', [('5', 0.0)])]

In [24]:
adj = filledMINGraph.collect()
adjDict = dict()
for i in adj:
    adjDict[i[0]] = i[1][1]
print(adjDict['1'])

[('1', 0), ('2', -3.0), ('3', inf), ('4', inf), ('5', 5.0)]


In [25]:
def newPath(vertice):
    known = vertice[1][0]
    edges = vertice[1][1][:]
    edges_rep = vertice[1][1]
    new = []
    for i in edges:
        newPath = i[1]+known
        new.append((i[0], (newPath, adjDict[i[0]])))
    return new



In [26]:
update = True
origin = filledMINGraph
new = filledMINGraph
flag=0
adjDictionary = getAdjDict(data, start)
while update:
    flag+=1
    origin = new
    
    newMAPGraph = new.flatMap(lambda x: newPath(x))
  #  print(newMAPGraph.collect())
   # print('sssssssss')
    new = newMAPGraph.reduceByKey(lambda x, y: x if x[0]<=y[0] else y)
    if new.collect() == origin.collect():
        update = False
    elif flag == len(adjDictionary)-1:
            update = False 
    new.collect()


NameError: name 'data' is not defined